In [1]:
import torch
import torch.nn.functional as F

In [18]:
torch.manual_seed(5)
pred = torch.randn(3, 5, requires_grad=True)        # 模拟预测结果，3个目标，共有5类，
print(pred)

tensor([[-0.4868, -0.6038, -0.5581,  0.6675, -0.1974],
        [ 1.9428, -1.4017, -0.7626,  0.6312, -0.8991],
        [-0.5578,  0.6907,  0.2225, -0.6662,  0.6846]], requires_grad=True)


In [19]:
# 模拟GT。 其中target1是target2的onehot形式。
# target1的形式，可以表示非互斥类别目标（即1目标可是多类别），
# 但是target2的形式表示的目标只能有一个类别。
target1 = torch.tensor([[1., 0., 0., 0., 0.],
                        [0., 0., 0., 1., 0.],
                        [0., 1., 0., 0., 0.]])
target2 = torch.tensor([0, 3, 1])

In [20]:
# 分别计算 pred的 sigmoid和softmax
sigmoid_pred = torch.sigmoid(pred)
softmax_pred = torch.softmax(pred,1)
print(sigmoid_pred)
print(softmax_pred)

tensor([[0.3807, 0.3535, 0.3640, 0.6609, 0.4508],
        [0.8747, 0.1975, 0.3181, 0.6528, 0.2892],
        [0.3641, 0.6661, 0.5554, 0.3393, 0.6648]], grad_fn=<SigmoidBackward>)
tensor([[0.1365, 0.1214, 0.1271, 0.4328, 0.1823],
        [0.6994, 0.0247, 0.0468, 0.1884, 0.0408],
        [0.0907, 0.3160, 0.1979, 0.0814, 0.3141]], grad_fn=<SoftmaxBackward>)


In [21]:
# 手写一遍公式，验证
import numpy as np
_sigmoid_pred = 1/(1+np.e**(-pred))                                # sigmoid公式
_softmax_pred = np.e**pred/(np.e**pred).sum(1,keepdim=True)        # softmax公式
print(_sigmoid_pred)
print(_softmax_pred)

tensor([[0.3807, 0.3535, 0.3640, 0.6609, 0.4508],
        [0.8747, 0.1975, 0.3181, 0.6528, 0.2892],
        [0.3641, 0.6661, 0.5554, 0.3393, 0.6648]], grad_fn=<MulBackward0>)
tensor([[0.1365, 0.1214, 0.1271, 0.4328, 0.1823],
        [0.6994, 0.0247, 0.0468, 0.1884, 0.0408],
        [0.0907, 0.3160, 0.1979, 0.0814, 0.3141]], grad_fn=<DivBackward0>)


In [27]:
# 用torch的函数计算3个CEloss
loss1 = F.binary_cross_entropy_with_logits(pred, target1, reduction='none')
loss2 = F.binary_cross_entropy(sigmoid_pred, target1,reduction='none')
loss3 = F.cross_entropy(pred, target2, reduction='none')
print(loss1)
print(loss2)
print(loss3)

tensor([[0.9659, 0.4361, 0.4525, 1.0816, 0.5993],
        [2.0767, 0.2201, 0.3829, 0.4265, 0.3414],
        [0.4526, 0.4063, 0.8105, 0.4145, 1.0930]],
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor([[0.9659, 0.4361, 0.4525, 1.0816, 0.5993],
        [2.0767, 0.2201, 0.3829, 0.4265, 0.3414],
        [0.4526, 0.4063, 0.8105, 0.4145, 1.0930]],
       grad_fn=<BinaryCrossEntropyBackward>)
tensor([1.9917, 1.6691, 1.1520], grad_fn=<NllLossBackward>)


In [30]:
# 手写celoss验证
_loss1=-target1*torch.log(sigmoid_pred)-(1-target1)*torch.log(1-sigmoid_pred)
_loss3=torch.sum(-target1*torch.log(softmax_pred),1)
print(_loss1)
print(_loss3)


tensor([[0.9659, 0.4361, 0.4525, 1.0816, 0.5993],
        [2.0767, 0.2201, 0.3829, 0.4265, 0.3414],
        [0.4526, 0.4063, 0.8105, 0.4145, 1.0930]], grad_fn=<SubBackward0>)
tensor([1.9917, 1.6691, 1.1520], grad_fn=<SumBackward1>)
